In [ ]:
from simesh import amr_loader
ds = amr_loader('/users/cpa/haow/codes/projects/fanspine_ffhd/qsl/data/fs_qsl0000.dat')
btotal = ds.mesh.udata[:,:,:,3:]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(btotal[50:350,50:350,0,2])

In [ ]:
# generate bfield for K-QSL code
import numpy as np
with open(r'/users/cpa/haow/codes/projects/fanspine_ffhd/qsl/data/bxyz.binary', 'wb') as file:
    b1f = btotal[...,0].T
    b2f = btotal[...,1].T
    b3f = btotal[...,2].T
    b1f.tofile(file)
    b2f.tofile(file)
    b3f.tofile(file)
data = np.fromfile('/users/cpa/haow/codes/projects/fanspine_ffhd/qsl/data/bxyz.binary', dtype=np.float64)
array = data.reshape((3,400,400,600)).T

In [ ]:
# generate vtk file for visualization
import pyvista as pv
xmin = ds.header['xmin']; xmax = ds.header['xmax']
nx, ny, nz = ds.header['domain_nx']

spacing = np.array(xmax-xmin) / np.array([nx, ny, nz])
origin = xmin + spacing/2
x = np.linspace(xmin[0],xmax[0],nx+1)
x = (x[1:]+x[:-1])/2
y = np.linspace(xmin[2],xmax[2],nz+1)
y = (y[1:]+y[:-1])/2

mesh = pv.ImageData(dimensions=(nx, ny, nz), spacing=spacing, origin=origin)
mesh['b1'] = btotal[...,0].T.reshape(-1)
mesh['b2'] = btotal[...,1].T.reshape(-1)
mesh['b3'] = btotal[...,2].T.reshape(-1)

mesh.save('/users/cpa/haow/codes/projects/fanspine_ffhd/qsl/data/bu.vtk')

## QSL analysis

In [ ]:
import os
import numpy as np
import f90nml
import matplotlib.pyplot as plt

os.chdir('/users/cpa/haow/codes/QSL/QSL_cartesian/mytest')

nml    = f90nml.read('par')
infile = nml['filename_par']['outfilename']
xstart = nml['cal_par']['x_start']
ystart = nml['cal_par']['y_start']
zstart = nml['cal_par']['z_start']
xend   = nml['cal_par']['x_end']
yend   = nml['cal_par']['y_end']
zend   = nml['cal_par']['z_end']
nlevel = nml['cal_par']['nlevel']

xdim   = nlevel*(xend-xstart) + 1
ydim   = nlevel*(yend-ystart) + 1
zdim   = nlevel*(zend-zstart) + 1

f    = open(infile,'rb')
data = np.fromfile(f, dtype=np.double, count=xdim*ydim*zdim*3)
q    = np.reshape(data,(3,zdim,ydim,xdim))

# a1=plt.subplot(131)
# plt.imshow(q[0,0,:,:],origin='lower')
# plt.title('sign(Bz)log(Q)')

# a2=plt.subplot(132)
# plt.imshow(q[1,0,:,:],origin='lower')
# plt.title('Field Line Length')

# a3=plt.subplot(133)
# plt.imshow(q[2,0,:,:],origin='lower')
# plt.title('Mask')

# plt.show()

from matplotlib.colors import LogNorm
plt.imshow(np.rot90(abs(q[0,:,20,:].T)), cmap='viridis', norm=LogNorm())
plt.colorbar()

In [ ]:
np.where(abs(q[0,:,:,:].T>0.8))

In [ ]:
idx = np.where(abs(q[0,:,:,:].T)>0.8)
xmin = np.array([-9, -4, 0])
xmax = np.array([3, 4, 8])
nx = 600; ny = 400; nz = 400
xp = np.linspace(xmin[0], xmax[0], nx+1)
xp = (xp[:-1]+xp[1:])/2
yp = np.linspace(xmin[1], xmax[1], ny+1)
yp = (yp[:-1]+yp[1:])/2
zp = np.linspace(xmin[2], xmax[2], nz+1)
zp = (zp[:-1]+zp[1:])/2
x0 = xp[100]; x1 = xp[299]
y0 = yp[180]; y1 = yp[219]
z0 = zp[50]; z1 = zp[149]
xp1 = np.linspace(x0, x1, 200); yp1 = np.linspace(y0, y1, 40); zp1 = np.linspace(z0, z1, 100)
n_samples = min(1000, len(idx[0]))
sample_indices = np.random.choice(len(idx[0]), size=n_samples, replace=False)
xq = xp1[idx[0][sample_indices]]
yq = yp1[idx[1][sample_indices]]
zq = zp1[idx[2][sample_indices]]
with open('/users/cpa/haow/codes/projects/fanspine_ffhd/points.txt', 'w') as f:
    for x, y, z in zip(xq, yq, zq):
        f.write(f'{x} {y} {z}\n')

In [ ]:
xp1[idx[0]]

## Use field lines

In [ ]:
import pyvista as pv
lines = '/users/cpa/haow/codes/projects/fanspine_ffhd/qsl/data/lines.vtk'
line_data = pv.read(lines)
points = line_data.points
#np.savetxt('/users/cpa/haow/codes/projects/fanspine_ffhd/data/line_points.csv', line_data.points, delimiter=' ', fmt='%.6f')

In [ ]:
mask1 = (abs(points[:,1])< 1) & (points[:,2]<0.02) & (points[:,0]<-7)
mask2 = (abs(points[:,1])< 1) & (points[:,2]<0.02) & (points[:,0]>-4)
fpl = points[mask1]
fpr = points[mask2]

In [ ]:
plt.scatter(fpl[:,0], fpl[:,2])

In [ ]:
# plot amr grids in special refine control
import yt
os.chdir('/users/cpa/haow/codes/projects/fanspine_ffhd')
datfile = 'data/fs_ffhd0000.dat'

ds = yt.load(datfile)

slc = yt.SlicePlot(ds, 'x', 'rho', origin='native')
slc.annotate_grids()
slc.annotate_streamlines('b2','b3', density=1, color='white', linewidth=1.2)
slc.show()